# creazione del modello che utilizza image encoder di SatCLIP

In [2]:
import torch
from torch import nn
import model

class SatCLIPClassifier(nn.Module):
    def __init__(self, num_classes, pretrained_model_path):
        super(SatCLIPClassifier, self).__init__()
        
        # Carica il modello SatCLIP pre-addestrato
        self.satclip = model.SatCLIP.load_from_checkpoint(pretrained_model_path)
        
        # Blocca i parametri del modello SatCLIP
        for param in self.satclip.parameters():
            param.requires_grad = False
        
        # Sblocca i parametri dell'encoder visivo
        for param in self.satclip.visual.parameters():
            param.requires_grad = True
        
        # Aggiungi un classificatore lineare
        self.classifier = nn.Linear(self.satclip.embed_dim, num_classes)

    def forward(self, images):
        # Codifica le immagini
        image_features = self.satclip.encode_image(images)
        
        # Normalizza le feature (come fatto in SatCLIP)
        image_features = image_features / image_features.norm(dim=1, keepdim=True)
        
        # Classifica
        outputs = self.classifier(image_features)
        return outputs

ModuleNotFoundError: No module named 'satmodel'

In [1]:
import satmodel
print(dir(satmodel))

ModuleNotFoundError: No module named 'satmodel'

# definire la classe per il dataset EuroSAT

In [9]:
import os
from PIL import Image
from torch.utils.data import Dataset

class EuroSATDataset(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.classes = sorted(os.listdir(root))
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        
        self.data = []
        self.targets = []
        
        for cls in self.classes:
            class_path = os.path.join(root, cls)
            for img_name in os.listdir(class_path):
                self.data.append(os.path.join(class_path, img_name))
                self.targets.append(self.class_to_idx[cls])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        img_path, target = self.data[index], self.targets[index]
        img = Image.open(img_path).convert('RGB')
        
        if self.transform:
            img = self.transform(img)
        
        return img, target

# dividere il dataset EuroSAT in train e test set

In [10]:
from torch.utils.data import DataLoader, random_split
from torchvision import transforms

# Impostazioni
eurosat_root = '/work/tesi_acastagni/EuroSAT'
batch_size = 64

# Preparazione del dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = EuroSATDataset(root=eurosat_root, transform=transform)
num_classes = len(dataset.classes)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# ottimizzazione

In [11]:
import torch
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pretrained_model_path = 'model'

# Creazione e preparazione del modello
model = SatCLIPClassifier(num_classes, pretrained_model_path).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

AttributeError: module 'model' has no attribute 'SatCLIP'

# training e test

In [ ]:
from tqdm import tqdm

def train_epoch(model, trainloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(trainloader, desc='Training'):
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    return running_loss / len(trainloader)

def evaluate(model, testloader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in tqdm(testloader, desc='Evaluating'):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    return accuracy

# loop

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    loss = train_epoch(model, trainloader, criterion, optimizer, device)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss:.4f}')
    
    accuracy = evaluate(model, testloader, device)
    print(f'Accuracy on test set: {accuracy:.2f}%')